In [9]:
import pandas as pd
import geopandas as gpd
import re
from shapely.geometry import Point
import matplotlib.pyplot as plt
## import seaborn as sns
import folium

In [10]:
pickup_report = pd.read_csv('data/trash_hauler_report.csv')

In [11]:
                               # Fill missing fields 
pickup_report['Trash Hauler'] = pickup_report['Trash Hauler'].fillna('N/A')
pickup_report['Trash Route'] = pickup_report['Trash Route'].fillna('N/A')
pickup_report['Trash Route'] = pickup_report['Trash Route'].fillna('0')

In [12]:
pickup_report['Zip Code'] = pickup_report['Zip Code'].fillna(0).astype('category')
assert pickup_report['Zip Code'].dtype == 'category'

print(pickup_report['Zip Code'].describe())

display(pickup_report['Zip Code'])

count     20226.0
unique       29.0
top       37013.0
freq       2278.0
Name: Zip Code, dtype: float64


0        37207.0
1        37218.0
2        37209.0
3        37207.0
4        37209.0
          ...   
20221    37013.0
20222    37206.0
20223    37214.0
20224    37013.0
20225    37217.0
Name: Zip Code, Length: 20226, dtype: category
Categories (29, float64): [0.0, 37013.0, 37027.0, 37076.0, ..., 37219.0, 37220.0, 37221.0, 37228.0]

In [14]:
# Clean up column names and Normalize data
pickup_report = pickup_report.rename(columns={'Request ': 'Request'})
pickup_report['Trash Hauler'] = pickup_report['Trash Hauler'].str.upper()

In [17]:
# Create a dataframe made up of ONLY 'missed' appearing in the 'Request" column
missedpu = pickup_report[pickup_report['Request'].str.contains(r'\bmissed\b', case=False, na=False, regex=True)]
print (missedpu.describe())

       Request Number  Council District  State Plan X  State Plan Y
count    15028.000000      14991.000000  1.500700e+04  1.500700e+04
mean    151361.267234         18.318524  1.766425e+06  6.592279e+05
std      71901.537542         10.107257  8.137032e+05  4.550526e+04
min      25274.000000          1.000000  1.663490e+06 -2.719106e+05
25%      87142.250000          8.000000  1.728654e+06  6.395148e+05
50%     148228.500000         20.000000  1.747701e+06  6.570097e+05
75%     220744.500000         27.000000  1.760562e+06  6.757087e+05
max     267137.000000         35.000000  3.496892e+07  2.204382e+06


In [16]:
## owed amounts